In [1]:
# 데이터 로드
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784', version=1)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
# train, test split
X_train = mnist['data'][:60000]
y_train = mnist['target'][:60000]
X_test = mnist['data'][60000:]
y_test = mnist['target'][60000:]

In [41]:
X_train.describe()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,60000.0,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.0000,60000.0,60000.0,60000.0,60000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200433,0.088867,0.045633,0.019283,0.015117,0.0020,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.042472,3.956189,2.839845,1.686770,1.678283,0.3466,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.0000,0.0,0.0,0.0,0.0


* 랜덤포레스트에 적용

In [9]:
# 랜덤포레스트에 적용 (n_estimators=10, random_state=42)
from sklearn.ensemble import RandomForestClassifier
rlf = RandomForestClassifier(n_estimators=10, random_state=42)
rlf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=42)

In [22]:
# 정확도 결과
y_pred = rlf.predict(X_test)
y_pred_proba = rlf.predict_proba(X_test)[:,1]
rlf.score(X_test, y_test)

0.9492

# 1. PCA

* PCA로 변환한 데이터 세트에 랜덤 포레스트 적용

In [16]:
# PCA 라이브러리 불러오기
from sklearn.decomposition import PCA

In [23]:
# PCA 적용 95% : n_components=0.95
pca = PCA(n_components=0.95)
pca.fit(X_train)

PCA(n_components=0.95)

In [24]:
train_img = pca.transform(X_train)
test_img = pca.transform(X_test)

print(f'train_img shape : {train_img.shape}')
print(f'test_img shape : {test_img.shape}')

train_img shape : (60000, 154)
test_img shape : (10000, 154)


In [26]:
# PCA 이후 랜덤포레스트 훈련
rlf.fit(train_img, y_train)

RandomForestClassifier(n_estimators=10, random_state=42)

In [27]:
# 정확도 결과 (X_test도 PCA 적용해야합니다.)
print('Training score : ', rlf.score(train_img, y_train))
print('Testing score  : ', rlf.score(test_img, y_test))

Training score :  0.9989833333333333
Testing score  :  0.9009


# 2. LDA

- LDA로 변환한 데이터 세트에 랜덤 포레스트 적용

In [28]:
# LDA 라이브러리 불러오기
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [33]:
# LDA 적용 : n_components=5
lda = LinearDiscriminantAnalysis(n_components=5)
mnist_lda=lda.fit_transform(X_train, y_train)

In [34]:
# LDA 이후 랜덤포레스트 훈련
rlf.fit(mnist_lda, y_train)

RandomForestClassifier(n_estimators=10, random_state=42)

In [37]:
# 정확도 결과 (X_test도 LDA 적용해야합니다.)
pred = rlf.predict(lda.transform(X_test))

In [39]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

0.8301